In [1]:
from espn_api.football import League
import pandas as pd
from flask import Flask, render_template
import threading
import time

In [2]:
# private league with cookies
league = League(league_id=637021, year=2024, espn_s2='AEBCCakbu%2B0%2FhbFeK5%2FgjfgBqJJfZKHfNjzHL2jCCx75d%2BXAUfjrRUGlUYOU%2BDcMyLnvZF9ASrpPFx%2Fd5IA4P8Yq1qMhcRE%2BqSa10zDy8NknbQWjzwKh3OVfI%2FCVZd2eKwMSzCNk54bD4FYRXMOMOVCp%2BwzXrZvHaoKs9nbe3Bsm%2BaKhCXOQ02AZbkrcGq%2B2naO9aSY3cXRoDjZaFgxYYcJnl7K23qiSoNPtt5MDZNjTeWFomxWJoC8Q84ob%2BrCve1L1ovlMK6Kg9KJ%2Br6UIYT2O', swid='{1BFA93C2-363A-4C34-A4BD-E6CE5E0C309B}')

In [3]:
# Flask App
app = Flask(__name__)

In [23]:
def get_team_scores_with_starters(league, week):
    """
    Fetch live scores and determine:
    1. Starters currently playing (not labeled 'BE').
    2. Starters yet to start their games (not labeled 'BE').
    If no players are found, return descriptive messages.
    """
    box_scores = league.box_scores(week=week)
    scores_and_players = []

    # Map team IDs to Team objects for easy lookup
    team_lookup = {team.team_id: team for team in league.teams}

    for game in box_scores:
        for team_id, lineup, score in [
            (game.home_team, game.home_lineup, game.home_score),
            (game.away_team, game.away_lineup, game.away_score)
        ]:
            # Resolve the team object from the team ID
            if isinstance(team_id, int):
                team = team_lookup.get(team_id)
            else:
                team = team_id  # Already a Team object

            # Handle case where team is None (unlikely but possible)
            team_name = team.team_name if team else "Unknown Team"

            # Filter players who are not on the bench
            currently_playing = [
                player.name for player in lineup
                if player.slot_position != "BE"  # Exclude bench players
                and player.points > 0  # Player is currently playing
                and player.game_played == 1  # Game is in progress
            ]

            not_started = [
                player.name for player in lineup
                if player.slot_position != "BE"  # Exclude bench players
                and player.game_played == 0  # Game has not started
            ]

            # Handle cases with no players in each category
            if not currently_playing:
                currently_playing = ["No players playing"]
            if not not_started:
                not_started = ["All players played"]

            # Add team data to the list
            scores_and_players.append({
                'Team Name': team_name,
                'Live Score': score,
                'Starters Currently Playing': ", ".join(currently_playing),
                'Starters Yet to Play': ", ".join(not_started)
            })

    return scores_and_players

In [24]:
week = 15  # Replace with the current week
scores_and_players = get_team_scores_with_starters(league, week)

# Print results
for entry in scores_and_players:
    print(entry)

{'Team Name': 'Enflamed Pancreases', 'Live Score': 1.7, 'Starters Currently Playing': 'No players playing', 'Starters Yet to Play': 'Garrett Wilson, Puka Nacua, Alvin Kamara, Brock Bowers, Jayden Daniels, Jaxon Smith-Njigba, Saquon Barkley, Cameron Dicker, Broncos D/ST'}
{'Team Name': 'Unknown Team', 'Live Score': 0, 'Starters Currently Playing': 'No players playing', 'Starters Yet to Play': 'All players played'}
{'Team Name': 'PartyLikeA GronkStar', 'Live Score': 0.0, 'Starters Currently Playing': 'No players playing', 'Starters Yet to Play': 'Breece Hall, Derrick Henry, Mike Evans, Josh Allen, David Njoku, Jameson Williams, Chuba Hubbard, Chris Boswell, Bengals D/ST'}
{'Team Name': 'Beermile Champ', 'Live Score': 2.3, 'Starters Currently Playing': 'No players playing', 'Starters Yet to Play': "Ja'Marr Chase, Nico Collins, Patrick Mahomes, Brian Robinson Jr., Jauan Jennings, Rhamondre Stevenson, Jonnu Smith, Matthew Wright, Packers D/ST"}
{'Team Name': 'Picks come 1st @anna🐐', 'Live S

In [15]:
for game in box_scores:
    print(type(game.home_team))
    print(game.home_team)
    break


<class 'espn_api.football.team.Team'>
Team(Team floop)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.13:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1